In [15]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np
import time
import datetime
import math
# import gurobipy as gp  # import the installed package




In [16]:
import sys
import gurobipy
print(f"python version: {sys.version}")
print(f"gurobipy version: {gurobipy.gurobi.version()}")

python version: 3.9.6 (default, Sep 26 2022, 11:37:49) 
[Clang 14.0.0 (clang-1400.0.29.202)]
gurobipy version: (11, 0, 0)


In [6]:
# config = {
#     'M': 1000000,
#     'lambda_for_G': 0.5,
#     'i_amount' :2, #2個客戶
#     'j_amount' :3, #三個設廠地點
#     'k_amount' :2, #兩種可分配資源（大車, 小車)
#     'l_amount' :0, #對手數量
#     'U_LT': [[3,5], [3,5], [3,5]],
#     'U_T': [5,10],
#     'U_L': [6,6,6],
#     'V': [[10,2], [2,1], [0,2]],
#     'H': [100, 50],
#     'D': [[1,1,1],[1,1,1]],
#     'D_comp':[],
#     'A_opponent_bar': [], #對手吸引力
#     'F': [0, 0, 0],
#     'C': [1, 1, 1], 
#     'B': [[1,1,1],[1,1,1],[1,1,1]],
# }
# config = { #0312成功版本config
#     'M': 1000000,
#     'lambda_for_G': 6e-6,
#     'i_amount' :1, #客戶
#     'j_amount' :1, #設廠地點
#     'k_amount' :1, #可分配資源（大車, 小車)
#     'l_amount' :1, #對手數量
#     'U_LT': [[3]],
#     'U_T': [5],
#     'U_L': [4],
#     'V': [[1000]],
#     'H': [100],
#     'D': [[1]],
#     'D_comp':[[1]],
#     'A_opponent_bar': [1000], #對手吸引力
#     'F': [0.1],
#     'C': [0.001], 
#     'B': [[0.1]],
# }
# config = {  # 2客戶 1設施
#     'M': 1000000,
#     'lambda_for_G': 6e-6,
#     'i_amount' :2, # 客戶
#     'j_amount' :1, # 設廠地點
#     'k_amount' :1, # 可分配資源（大車, 小車)
#     'l_amount' :1, # 對手數量
#     'U_LT': [[2]], # Max amount of resource k allowed to be allocated to facility j, Dim : (j, k)
#     'U_T': [3],  # Max allocation limit for resource k, Dim : (k)
#     'U_L': [3],  # Max capacity for facility j, Dim : (j)
#     'V': [[500]],
#     'H': [100, 5000],
#     'D': [[2], [1]],
#     'D_comp':[[1], [1]],
#     'A_opponent_bar': [100], #對手吸引力
#     'F': [1],
#     'C': [0.02], 
#     'B': [[0.1]],
# }

In [10]:
'''
Instance Parameters
===============================
Customer Location: I
Facility Location: J
Resource Type: K
Competitor Facility Location: L

U_LT : Max amount of resource k allowed to be allocated to facility j, Dim : (j, k)
U_T : Max allocation limit for resource k, Dim : (k)
U_L : Max capacity for facility j, Dim : (j)
V : Attractiveness yield of resource k allocated to facility j, Dim : (j, k)
H : Max demand for customer i, Dim : (i)
D : Distance between customer i and facility j , Dim : (i, j)
D_comp : Distance between customer i and competitor l, Dim : (i, l)
F : Fixed cost for building facility j, Dim : (j)
C : Cost per unit of extra attractiveness at facility j, Dim : (j)
B : Cost of allocating a unit of resource k to facility j, Dim : (j, k)
===============================
'''
config = {  # 2客戶 1設施
    'M': 1000,
    'lambda_for_G': 6e-6,
    'i_amount' :2, # 客戶
    'j_amount' :1, # 設廠地點
    'k_amount' :1, # 可分配資源（大車, 小車)
    'l_amount' :1, # 對手數量
    'U_LT': [[2]], # Max amount of resource k allowed to be allocated to facility j, Dim : (j, k)
    'U_T': [3],  # Max allocation limit for resource k, Dim : (k)
    'U_L': [3],  # Max capacity for facility j, Dim : (j)
    'V': [[5000]],
    'H': [100, 5000],
    'D': [[2], [1]],
    'D_comp':[[1], [1]],
    'A_opponent_bar': [100], #對手吸引力
    'F': [1],
    'C': [0.02], 
    'B': [[0.1]],
}


I = set(range(0, config['i_amount']))  
J = set(range(0, config['j_amount']))  
K = set(range(0, config['k_amount']))  
L = set(range(0, config['l_amount']))  
'''
Decision Variables
===============================
y : Whether location j build facility or not, Dim: (j) , BINARY
x : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS
aX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS
===============================

Other Variables
u : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS
P : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS
TA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS
===============================
'''

'\nDecision Variables\n===============================\ny : Whether location j build facility or not, Dim: (j) , BINARY\nx : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS\naX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS\n===============================\n\nOther Variables\nu : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS\nP : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS\nTA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS\n===============================\n'

In [11]:
# Initialize the model
model = Model("Competitive Facility Location")
model.setParam('NumericFocus', 3)
y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="utility(u)", lb=0) # utility var(3.5)
P = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="P, utility vs TA ratio", lb=0.0, ub=1.0) # Probability of i choosing j
TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000001)
TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="negative TAi*lambda(e's power)", lb= -100000, ub= -0.000001) #-lambda*TAi
G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars", lb=0.001, ub=0.999) #e^-(lambda*TAi), ub=0.5

model.update()

model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

model.addConstrs((u[i, j] == ((quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j]) / (config['D'][i][j]**2)) for i in I for j in J), "(3.5) utility for facility j to customer i")
model.addConstrs((u[i, j] == P[i, j] * TA[i] for i in I for j in J), "(3.8) calculate P by uij/TAi, 分母要是常數所以用乘的")
## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + (quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) for i in I), "(3.7) calculate total TA by u and A_bar")

## G的計算
# def G(tai):
#     return 1 - np.exp(-config['lambda_for_G'] * tai)
model.addConstrs((TAi_lambda[i] == (-config['lambda_for_G'] * TA[i]) for i in I), "negative tai*lambda constr")
# Gurobi無法直接在目標式有exponential
for i in I:
    model.addGenConstrExpA(TAi_lambda[i], G_exp_vars[i], math.e, name=f"G_exp_constr_{i}")



# Objective function: Maximize profit by attracting customers - costs
objective = ((quicksum(config['H'][i] * (1 - G_exp_vars[i]) * (quicksum(P[i, j] for j in J)) for i in I)) \
          - (quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J)))
model.setObjective(objective, GRB.MAXIMIZE)


# Solve and Output
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
else:
    print("No optimal solution found.")
    model.computeIIS()
    # 将IIS输出到文件
    # model.write("model_iis.mps")
    print("IIS written to file 'model_iis.mps'")



Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.5.0 21F79)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 13 columns and 20 nonzeros
Model fingerprint: 0xc0780acd
Model has 2 quadratic objective terms
Model has 2 quadratic constraints
Model has 2 general constraints
Variable types: 12 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [6e-06, 5e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e-02, 5e+03]
  QObjective range [2e+02, 1e+04]
  Bounds range     [1e-08, 1e+05]
  RHS range        [3e+00, 1e+02]
Presolve added 0 rows and 46 columns
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 22 rows, 63 columns, 190 nonzeros
Presolved model has 2 SOS constraint(s)
Presolved model has 4 bilinear constraint(s)
         in pr

In [12]:
for var in model.getVars():
    print(f"{var.varName} = {var.x}")
print("")

y[0] = 1.0
x[0,0] = 2.0
aX[0] = 1000.0
utility(u)[0,0] = 2750.0
utility(u)[1,0] = 11000.0
P, utility vs TA ratio[0,0] = 0.964912280701767
P, utility vs TA ratio[1,0] = 0.9909909909910001
TA[0] = 2850.0
TA[1] = 11100.0
negative TAi*lambda(e's power)[0] = -0.0171
negative TAi*lambda(e's power)[1] = -0.0666
G_exp_vars[0] = 0.9833705388527908
G_exp_vars[1] = 0.9365016152132405



# References

## G calculation substitude exponential

https://www.gurobi.com/documentation/10.0/refman/py_model_agc_exp.html

https://support.gurobi.com/hc/en-us/community/posts/360077178491-Build-an-objective-function-with-Log-and-Exponential-inside-

https://support.gurobi.com/hc/en-us/community/posts/10481250723217-How-to-add-log-and-exponential-term-in-objective-function
